In [ ]:

%reset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

'''
Hyper-parameters Settings
'''
HAMPEL_WINDOW_SIZE = 21
HAMPEL_THRESHOLD = 1
HAMPEL_IMPUTATION = True

'''
Path Settings
'''
CONFIGURATION_FILE_PATH = "./data/train/data_config.csv"
DATASET_PATH = "./data/train/"

'''
Program Configurations
'''
COP_OUTPUT_SAVE_IMAGE = False
COP_OUTPUT_SAVE_CSV = True
LOADCELL_OUTPUT_SAVE_IMAGE = False
LOADCELL_OUTPUT_SAVE_CSV = True
FSR_SCALEUP_OUTPUT_SAVE_IMAGE = True
FSR_SCALEUP_COMPARE_SAVE_IMAGE = False
FSR_REGION_CROP_ENABLE = True
DYNAMIC_FSR_SCALEUP = True


'''
Figure Settings
'''
pd.set_option('display.width', 200) # for display width



'''
Read Configurration File (only xls)
'''
data_config = pd.read_csv(CONFIGURATION_FILE_PATH, header=0, index_col=0)
print("Configuration Dataframe dimension: ", data_config.shape)


'''
Read all FSR matrix data and Seat sensor data
'''
fsr_dataframe = {}
seat_dataframe = {}

for idx in data_config.index:
    fsr_filepath = DATASET_PATH+data_config.loc[idx, "fsr_matrix_1d_datafile"] # set FSR matrix data filepath
    seat_filepath = DATASET_PATH+data_config.loc[idx, "seat_datafile"] # set Seat data filepath
    print(idx, ") read data files : ", fsr_filepath, ",", seat_filepath)

    fsr_dataframe[idx] = pd.read_csv(fsr_filepath, header=0, index_col=False).iloc[:,0:162] # read FSR matrix data file
    seat_dataframe[idx] = pd.read_csv(seat_filepath, header=0, index_col=False) # read Seat data file

    # clear unnecessary columns
    del seat_dataframe[idx]['Measurement time'] # remove unnecessary column
    del fsr_dataframe[idx]['Measurement Time (sec)'] # remove unnecessary column

'''
Preproceess : Data Segmentation by mtime
   - @brief     FSR matrix data and Seat data should be segmented by mtime
   - @output    segmented dataframes
'''

# output dict.
fsr_dataframe_standard_segment = {}
fsr_dataframe_relax_segment = {}
seat_loadcell_dataframe_standard_segment = {}
seat_loadcell_dataframe_relax_segment = {}


for idx in data_config.index:
    mtime = data_config.loc[idx, ['standard_s_mtime', "standard_e_mtime", "relax_s_mtime", "relax_e_mtime"]]

    # seat loadcell segmentation
    seat_loadcell_dataframe_standard_segment[idx] = seat_dataframe[idx][(seat_dataframe[idx]['mtime']>=mtime.standard_s_mtime) & (seat_dataframe[idx]['mtime']<=mtime.standard_e_mtime)]
    seat_loadcell_dataframe_relax_segment[idx] = seat_dataframe[idx][(seat_dataframe[idx]['mtime']>=mtime.relax_s_mtime) & (seat_dataframe[idx]['mtime']<=mtime.relax_e_mtime)]

    # fsr matrix segmentation
    fsr_dataframe_standard_segment[idx] = fsr_dataframe[idx][(fsr_dataframe[idx]['mtime']>=mtime.standard_s_mtime) & (fsr_dataframe[idx]['mtime']<=mtime.standard_e_mtime)]
    fsr_dataframe_relax_segment[idx] = fsr_dataframe[idx][(fsr_dataframe[idx]['mtime']>=mtime.relax_s_mtime) & (fsr_dataframe[idx]['mtime']<=mtime.relax_e_mtime)]

    print("FSR Segments@Standard size : ", len(fsr_dataframe_standard_segment[idx]), ", FSR Segments@Relax size : ", len(fsr_dataframe_relax_segment[idx]))
    print("Seat Segments@Standard size : ", len(seat_loadcell_dataframe_standard_segment[idx]), ", Seat Segments@Relax size : ", len(seat_loadcell_dataframe_relax_segment[idx]))



'''
Preproceess : Segmented FSR Matrix Data up-scaling
   - @brief raw data(16x10 dim.) build high resolution image with interpolation(super-resolution)
   - @method    catrom (Centripetal Catmull–Rom spline)
   - @output    scaled up images for all swquences
   - @note  1) saving output image may take too long time. if you already done, skip this code block.
            2) converting all data to image is not efficient. use start/end time configuration defined in data_config.csv
            3) if file is already exist, process will be skiped.
'''
import os
import os.path
import gc
from skimage import io, color


if FSR_SCALEUP_OUTPUT_SAVE_IMAGE==True:
    try:
        os.mkdir("./images/fsr_matrix") # create diretory
    except FileExistsError:
        pass

fsr_standard_interpolated_paths = {}
fsr_relax_interpolated_paths = {}
fsr_crop_standard_interpolated_paths = {}
fsr_crop_relax_interpolated_paths = {}

for idx in data_config.index:
    fsr_standard_segment_1d = fsr_dataframe_standard_segment[idx].iloc[:,1:161]
    fsr_standard_segment_2d = fsr_standard_segment_1d.values.reshape(-1, 16, 10) # reshape

    fsr_relax_segment_1d = fsr_dataframe_relax_segment[idx].iloc[:,1:161]
    fsr_relax_segment_2d = fsr_relax_segment_1d.values.reshape(-1, 16, 10)

    try:
        os.mkdir("./images/fsr_matrix/{}".format(idx)) # create diretory for each id
    except FileExistsError:
        pass
    
    # @standard mode
    standard_fsr_file_list = []
    standard_fsr_crop_file_list = []
    for ridx in range(fsr_standard_segment_2d.shape[0]):
        result_image_filepath = "./images/fsr_matrix/{}/standard_{}.jpg".format(idx, ridx)
        result_crop_image_filepath = "./images/fsr_matrix/{}/standard_crop_{}.jpg".format(idx, ridx)

        if FSR_SCALEUP_OUTPUT_SAVE_IMAGE==True:
            if os.path.isfile(result_image_filepath)==True: # file exist
                print("{} is already exist".format(result_image_filepath))
            else:
                fig = plt.figure()
                plt.axis('off')
                if DYNAMIC_FSR_SCALEUP==True:
                    plt.imshow(fsr_standard_segment_2d[ridx], interpolation='catrom', cmap='Greys_r')
                else:
                    plt.imshow(fsr_standard_segment_2d[ridx], interpolation='catrom', vmin=0, vmax=255, cmap='Greys_r')
                plt.savefig(result_image_filepath, bbox_inches='tight', pad_inches=0)
                plt.close(fig)

            if FSR_REGION_CROP_ENABLE==True: # crop
                if os.path.isfile(result_crop_image_filepath)==True: # file exist
                    print("{} is already exist".format(result_crop_image_filepath))
                else:
                    image = io.imread(result_image_filepath)
                    grayscale = color.rgb2gray(image)
                    crop = grayscale[0:grayscale.shape[0],int(grayscale.shape[1]/2):grayscale.shape[1]]
                    io.imsave(result_crop_image_filepath, crop)
                    print("crop image saved : {}".format(result_crop_image_filepath))
                    del image
                standard_fsr_crop_file_list.append(result_crop_image_filepath)
        
        standard_fsr_file_list.append(result_image_filepath)
        print("(standard) saved output images for id {}, {}".format(idx, ridx))
        
    fsr_standard_interpolated_paths[idx] = pd.DataFrame(standard_fsr_file_list, columns=['path'])
    fsr_crop_standard_interpolated_paths[idx] = pd.DataFrame(standard_fsr_crop_file_list, columns=['path'])
    

    # @ relax mode
    relax_fsr_file_list = []
    relax_fsr_crop_file_list = []
    for ridx in range(fsr_relax_segment_2d.shape[0]):
        result_image_filepath = "./images/fsr_matrix/{}/relax_{}.jpg".format(idx, ridx)
        result_crop_image_filepath = "./images/fsr_matrix/{}/relax_crop_{}.jpg".format(idx, ridx)

        if FSR_SCALEUP_OUTPUT_SAVE_IMAGE==True:
            if os.path.isfile(result_image_filepath)==True: # file exist
                print("{} is already exist".format(result_image_filepath))
            else:
                fig = plt.figure()
                plt.axis('off')
                if DYNAMIC_FSR_SCALEUP==True:
                    plt.imshow(fsr_relax_segment_2d[ridx], interpolation='catrom', cmap='Greys_r')
                else:
                    plt.imshow(fsr_relax_segment_2d[ridx], interpolation='catrom', vmin=0, vmax=255, cmap='Greys_r')
                plt.savefig(result_image_filepath, bbox_inches='tight', pad_inches=0)
                plt.close(fig)

            if FSR_REGION_CROP_ENABLE==True: # crop
                if os.path.isfile(result_crop_image_filepath)==True: # file exist
                    print("{} is already exist".format(result_crop_image_filepath))
                else:
                    image = io.imread(result_image_filepath)
                    grayscale = color.rgb2gray(image)
                    crop = grayscale[0:grayscale.shape[0],int(grayscale.shape[1]/2):grayscale.shape[1]]
                    io.imsave(result_crop_image_filepath, crop)
                    print("crop image saved : {}".format(result_crop_image_filepath))
                    del image
                relax_fsr_crop_file_list.append(result_crop_image_filepath)
        
        relax_fsr_file_list.append(result_image_filepath)
        print("(relax) saved output images for id {}, {}".format(idx, ridx))
        
    fsr_relax_interpolated_paths[idx] = pd.DataFrame(relax_fsr_file_list, columns=['path'])
    fsr_crop_relax_interpolated_paths[idx] = pd.DataFrame(relax_fsr_crop_file_list, columns=['path'])


    if FSR_SCALEUP_COMPARE_SAVE_IMAGE==True:
        # show compared graph between raw and upscaled data
        # @ standard mode
        for ridx in range(fsr_standard_segment_2d.shape[0]):

            if DYNAMIC_FSR_SCALEUP==True:
                result_image_filepath = "./images/fsr_matrix/{}/compared_standard_dynamic_{}.png".format(idx, ridx)
            else:
                result_image_filepath = "./images/fsr_matrix/{}/compared_standard_static_{}.png".format(idx, ridx)

            # static interpolation
            if os.path.isfile(result_image_filepath)==True: # file exist
                print("{} is already exist".format(result_image_filepath))
            else:

                fig = plt.figure(figsize=(11,6), constrained_layout=True)

                # raw matrix data plot
                fig_raw = fig.add_subplot(1,2,1)
                if DYNAMIC_FSR_SCALEUP==True:
                    ax_raw = fig_raw.imshow(fsr_standard_segment_2d[ridx], interpolation='None', cmap='viridis')
                    fig_raw.set_title("Raw(Dynamic) FSR Matrix Data", fontsize=16)
                else:
                    ax_raw = fig_raw.imshow(fsr_standard_segment_2d[ridx], interpolation='None', vmin=0, vmax=255, cmap='viridis')
                    fig_raw.set_title("Raw(Static) FSR Matrix Data", fontsize=16)
                fig_raw.set_xlabel('X position(mm)')
                fig_raw.set_ylabel('Y position(mm)')
                fig.colorbar(ax_raw)

                # interpolated matrix data plot
                fig_hr = fig.add_subplot(1,2,2)
                if DYNAMIC_FSR_SCALEUP==True:
                    ax_hr = fig_hr.imshow(fsr_standard_segment_2d[ridx], interpolation='catrom', cmap='viridis')
                    fig_hr.set_title("Interpolated(Dynamic) FSR Matrix Data", fontsize=16)
                else:
                    ax_hr = fig_hr.imshow(fsr_standard_segment_2d[ridx], interpolation='catrom', vmin=0, vmax=255, cmap='viridis')
                    fig_hr.set_title("Interpolated(Static) FSR Matrix Data", fontsize=16)
                
                fig_hr.set_xlabel('X position(mm)')
                fig_hr.set_ylabel('Y position(mm)')
                fig.colorbar(ax_hr)
            
                fig.savefig(result_image_filepath)
                print("(standard) save to image : ",result_image_filepath)
                plt.close(fig)

        # @relax mode
        for ridx in range(fsr_relax_segment_2d.shape[0]):
            # save to image file
            if DYNAMIC_FSR_SCALEUP==True:
                result_image_filepath = "./images/fsr_matrix/{}/compared_relax_dynamic_{}.png".format(idx, ridx)
            else:
                result_image_filepath = "./images/fsr_matrix/{}/compared_relax_static_{}.png".format(idx, ridx)

            if os.path.isfile(result_image_filepath)==True: # file exist
                print("{} is already exist".format(result_image_filepath))
            else:

                fig = plt.figure(figsize=(11,6), constrained_layout=True)

                # raw matrix data plot
                fig_raw = fig.add_subplot(1,2,1)
                if DYNAMIC_FSR_SCALEUP==True:
                    ax_raw = fig_raw.imshow(fsr_relax_segment_2d[ridx], interpolation='None', cmap='viridis')
                    fig_raw.set_title("Raw(Dynamic) FSR Matrix Data", fontsize=16)
                else:
                    ax_raw = fig_raw.imshow(fsr_relax_segment_2d[ridx], interpolation='None', vmin=0, vmax=255, cmap='viridis')
                    fig_raw.set_title("Raw(Static) FSR Matrix Data", fontsize=16)
                fig_raw.set_xlabel('X position(mm)')
                fig_raw.set_ylabel('Y position(mm)')
                fig.colorbar(ax_raw)

                # interpolated matrix data plot
                fig_hr = fig.add_subplot(1,2,2)
                if DYNAMIC_FSR_SCALEUP==True:
                    ax_hr = fig_hr.imshow(fsr_relax_segment_2d[ridx], interpolation='catrom', cmap='viridis')
                    fig_hr.set_title("Interpolated(Dynamic) FSR Matrix Data", fontsize=16)
                else:
                    ax_hr = fig_hr.imshow(fsr_relax_segment_2d[ridx], interpolation='catrom', vmin=0, vmax=255, cmap='viridis')
                    fig_hr.set_title("Interpolated(Static) FSR Matrix Data", fontsize=16)
                fig_hr.set_xlabel('X position(mm)')
                fig_hr.set_ylabel('Y position(mm)')
                fig.colorbar(ax_hr) 
            
                fig.savefig(result_image_filepath)
                print("(relax) save to image : ",result_image_filepath)
                plt.close(fig)



'''
Model Training
@brief  Multiclass Classification using CONV+GRU with keras
'''
import random

# class separation (positive, negative)
shuffled_index = np.array(data_config.index)
random.shuffle(shuffled_index)
positive_class = shuffled_index[0:5] # first 5 index select from shuffled_index
negative_class = shuffled_index[5:]
negative_class_zero = np.zeros(len(negative_class), dtype=int)
classes = np.append(positive_class, 0)

# 1. input image transfer into the convolution network
from keras.layers import Conv2D, BatchNormalization, MaxPool2D, GlobalMaxPool2D
import keras
import numpy as np
from keras_video import VideoFrameGenerator
import glob
import os

# user parameters
SIZE = (68, 217) # crop image
CHANNELS = 3 # image channels
NBFRAME = 5 # near frames
BS = 10 #batch size
KERNEL_SIZE = (3,3)
EPOCHS=20

def build_convnet(shape=(SIZE, CHANNELS)):
    momentum = 0.9
    model = keras.Sequential()
    model.add(Conv2D(64, KERNEL_SIZE, input_shape=shape, padding='same', activation='relu')) # shape = (row, col, channels)
    model.add(Conv2D(64, KERNEL_SIZE, padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPool2D())

    model.add(Conv2D(128, KERNEL_SIZE, padding='same', activation='relu'))
    model.add(Conv2D(128, KERNEL_SIZE, padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPool2D())

    model.add(Conv2D(256, KERNEL_SIZE, padding='same', activation='relu'))
    model.add(Conv2D(256, KERNEL_SIZE, padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPool2D())

    model.add(Conv2D(512, KERNEL_SIZE, padding='same', activation='relu'))
    model.add(Conv2D(512, KERNEL_SIZE, padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPool2D())

    # flatten
    model.add(GlobalMaxPool2D())
    return model

from keras.layers import TimeDistributed, GRU, Dense, Dropout, LSTM

def action_model(shape=(SIZE, CHANNELS), nbout=5):
    #create convnet with (112, 112, 1) input shape
    convnet = build_convnet(shape[1:])

    #then create out final model
    model = keras.Sequential()

    # add the convnet with (5, 112, 112, 3) shape
    model.add(TimeDistributed(convnet, input_shape=shape))

    # you can also use GRU or LSTM
    model.add(GRU(5))

    # finally we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nbout, activation='softmax'))

    return model


# model compile
INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,)
print(INSHAPE)
model = action_model(INSHAPE, len(classes))
#optimizer = keras.optimizers.Adam(0.001)
optimizer = keras.optimizers.SGD(lr=0.01)
model.compile(
    optimizer,
    'categorical_crossentropy',
    metrics=['acc']
)

# use sub directories names as classes
classes = [i.split(os.path.sep)[1] for i in glob.glob('train/*')]
classes.sort()

# pattern to get videos and classes
glob_pattern='train/{classname}/standard_*.avi'
# for data augmentation
# data_aug = keras.preprocessing.image.ImageDataGenerator(
#     zoom_range=.1,
#     horizontal_flip=False,
#     rotation_range=8,
#     width_shift_range=.2,
#     height_shift_range=.2)

# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=.2, 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    #transformation=data_aug,
    use_frame_cache=False)


valid = train.get_validation_generator()
print(valid)

# create a "chkp" directory before to run that
# because ModelCheckpoint will write models inside
callbacks = [
    keras.callbacks.ReduceLROnPlateau(verbose=1),
    keras.callbacks.ModelCheckpoint(
        'chkp/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
        verbose=1),
]

history = model.fit(
    train,
    validation_data=valid,
    verbose=1,
    epochs=EPOCHS,
    callbacks=callbacks
)


